## Создание базы данных и таблиц

In [29]:
import csv
import pandas as pd
from datetime import datetime

Чтение данных из csv-файлов

In [11]:
with open('P9-Pets.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    pets = []
    for row in spamreader:
        pets.append(row)

In [12]:
with open('P9-Owners.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    owners = []
    for row in spamreader:
        owners.append(row)

In [13]:
with open('P9-ProceduresHistory.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    history = []
    for row in spamreader:
        history.append(row)

In [14]:
with open('P9-ProceduresDetails.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    details = []
    for row in spamreader:
        details.append(row)

Подключение к mysql и создание базы

In [2]:
# import mysql.connector
import mysql.connector as sql

In [3]:
# con = mysql.connector.connect(host='127.0.0.1', port=3306, user='root', password='kat_dreaming99')
con = sql.connect(host='127.0.0.1', port=3306, user='root', password='Sunny_Flower_35')

In [4]:
cur=con.cursor(buffered=True, dictionary=True)

In [5]:
cur.execute("CREATE DATABASE IF NOT EXISTS vet_database /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci */ /*!80016 DEFAULT ENCRYPTION='N' */;")

In [6]:
# con = mysql.connector.connect(host='127.0.0.1', port=3306, database='vet_database', user='root', password='kat_dreaming99')
con = sql.connect(host='127.0.0.1', port=3306, database='vet_database', user='root', password='Sunny_Flower_35')

In [7]:
cur = con.cursor(buffered=True, dictionary=True)

Создание и заполнение таблиц

In [25]:
cur.execute("""CREATE TABLE IF NOT EXISTS pets (
  id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
  pet_id VARCHAR(24) NOT NULL,
  name VARCHAR(24) NOT NULL,
  kind VARCHAR(24) NOT NULL,
  gender VARCHAR(24) BINARY,
  age INT,
  owner_id VARCHAR(24))
""")

In [26]:
for i, row in enumerate(pets[1:]):
    try:
        sss = f"INSERT INTO pets (pet_id, name, kind, gender, age, owner_id) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}')"
        cur.execute(sss)
    except:
        print("Error - INSERT INTO pets (pet_id, name, kind, gender, age, owner_id) VALUES ('"+row[0]+"', '"+row[1]+"', '"+row[2]+"', '"+row[3]+"', '"+row[4]+"', '"+row[5]+"')")
    
    con.commit()

In [27]:
cur.execute("""CREATE TABLE IF NOT EXISTS owners (
  id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
  owner_id VARCHAR(24) NOT NULL,
  name VARCHAR(24) NOT NULL,
  surname VARCHAR(24) NOT NULL,
  address VARCHAR(24) NOT NULL,
  city VARCHAR(24) NOT NULL,
  state VARCHAR(24) NOT NULL,
  state_full VARCHAR(24) NOT NULL,
  zipcode VARCHAR(24))
""")

In [31]:
for row in owners[1:]:
    try:
        sss = f"INSERT INTO owners (owner_id, name, surname, address, city, state, state_full, zipcode) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}', '{row[7]}')"
        cur.execute(sss)
    except:
        print("Error - INSERT INTO owners (owner_id, name, surname, address, city, state, state_full, zipcode) VALUES ('"+row[0]+"', '"+row[1]+"', '"+row[2]+"', '"+row[3]+"', '"+row[4]+"', '"+row[5]+"', '"+row[6]+"', '"+row[7]+"')")
    
    con.commit()

In [32]:
cur.execute("""CREATE TABLE IF NOT EXISTS history (
  id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
  pet_id VARCHAR(24) NOT NULL,
  date DATETIME NOT NULL,
  type VARCHAR(24) NOT NULL,
  subcode VARCHAR(24) NOT NULL)
""")

In [33]:
for i, row in enumerate(history[1:]):
    try:
        sss = f"INSERT INTO history (pet_id, date, type, subcode) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}')"
        cur.execute(sss)
    except:
        print("Error - INSERT INTO history (pet_id, date, type, subcode) VALUES ('"+row[0]+"', '"+row[1]+"', '"+row[2]+"', '"+row[3]+"')")
    
    con.commit()

In [34]:
cur.execute("""CREATE TABLE IF NOT EXISTS details (
  id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
  type VARCHAR(24) NOT NULL,
  subcode VARCHAR(24) NOT NULL,
  description VARCHAR(24) NOT NULL,
  price INT NOT NULL)
""")

In [35]:
for i, row in enumerate(details[1:]):
    try:
        sss = f"INSERT INTO details (type, subcode, description, price) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}')"
        cur.execute(sss)
    except:
        print("Error - INSERT INTO details (type, subcode, description, price) VALUES ('"+row[0]+"', '"+row[1]+"', '"+row[2]+"', '"+row[3]+"')")
    
    con.commit()

В двух таблицах (history и details) хранение данных показалось нам некорректным: для определения типа процедуры использовалось два признака - Type (общий тип, например, "Grooming") и SubCode (более конкретный тип, например, "01" как процедура, включающая мойку). Мы добавили в таблицу записи процедур признак "proc_id", содержащий информацию об id процедуры (как объединение пары описанных признаков) из таблицы details.

In [9]:
good_id = """ALTER TABLE vet_database.history ADD COLUMN proc_id INT DEFAULT 0"""
upd_prod = """UPDATE vet_database.history t1
INNER JOIN vet_database.details t2 
ON t1.type = t2.type AND t1.subcode = t2.subcode
SET t1.proc_id = t2.id"""

try:
    cur.execute(good_id)
except:
    print('Column already exists.')
con.commit()

Column already exists.


In [10]:
try:
    cur.execute(upd_prod)
except:
    print('All is already up-to-date')
con.commit()

All is already up-to-date


In [11]:
cur.execute("SELECT * FROM history")
res = cur.fetchone()
for k, v in res.items():
    print(k, v, sep=': ')

id_proc: 1
pet_id: A8-1181
date: 2016-01-10 00:00:00
proc_id: 11


## Интерфейс

### Добавление в таблицу

In [12]:
choice = input('Выберите таблицу, в которую хотите добавить данные: pets (наберите 1), owners (2), details (3): ').lower()
if choice == '1' or choice == 'pets':
    table = 'pets'
    print('Вы выбрали таблицу', table)
elif choice == '2' or choice == 'owners':
    table = 'owners'
    print('Вы выбрали таблицу', table)
elif choice == '3' or choice == 'details':
    table = 'details'
    print('Вы выбрали таблицу', table)
else:
    print('Вы ввели некорректный номер таблицы, попробуйте запустить заново.')

Выберите таблицу, в которую хотите добавить данные: pets (наберите 1), owners (2), details (3): pets
Вы выбрали таблицу pets


In [13]:
get_cols = 'SHOW columns FROM ' + table
cur.execute(get_cols)
columns = ([d['Field'] for d in cur.fetchall()])
print('Сейчас вам нужно будет заполнить данные о каждом из полей: ' + ', '.join(columns[1:]) + '.')
row = []
for column in columns[1:]:
    row.append(input('Введите данные о ' + column + ': '))

if table == 'pets':
    add = f"INSERT INTO pets (pet_id, name, kind, gender, age, owner_id) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}')"
elif table == 'owners':
    add = f"INSERT INTO owners (owner_id, name, surname, address, city, state, state_full, zipcode) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}', '{row[7]}')"
else:
    add = f"INSERT INTO details (type, subcode, description, price) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}')"
    
try:
    cur.execute(add)
except:
    print("Что-то пошло не так. Проверьте запрос и попробуйте ещё раз.")
con.commit()

Сейчас вам нужно будет заполнить данные о каждом из полей: pet_id, name, kind, gender, age, owner_id.
Введите данные о pet_id: 098014
Введите данные о name: nod
Введите данные о kind: cat
Введите данные о gender: iwf
Введите данные о age: k
Введите данные о owner_id: 2
Что-то пошло не так. Проверьте запрос и попробуйте ещё раз.


### Удаление данных

In [15]:
choice = input('Выберите таблицу, из которой хотите удалить данные: pets (наберите 1), owners (2): ').lower()
if choice == '1' or choice == 'pets':
    table = 'pets'
    print('Вы выбрали таблицу', table)
elif choice == '2' or choice == 'owners':
    table = 'owners'
    print('Вы выбрали таблицу', table)
else:
    print('Вы ввели некорректный номер таблицы, попробуйте запустить заново.')

Выберите таблицу, из которой хотите удалить данные: pets (наберите 1), owners (2): pets
Вы выбрали таблицу pets


In [17]:
get_cols = 'SHOW columns FROM ' + table
cur.execute(get_cols)
columns = ([d['Field'] for d in cur.fetchall()])
print('Сейчас вам нужно будет выбрать, по каким полям из ' + ', '.join(columns[1:]) + ' нужно будет искать данные (не более трёх). Перечислите названия через запятую, пожалуйста.')
cols = input('По каким полям? ')
cols = cols.split(',')
cols = [col.strip() for col in cols]

raw = []
if len(cols) > 3 or len(cols) < 1:
    print('Выберите, пожалуйста, одно, два или три поля для поиска и запустите заново.')
else:
    for column in cols:
        row.append("'"+input('Введите значения параметра ' + column + ': ')+"'")
    if len(cols) == 1:
        d = "DELETE FROM "+table+" WHERE "+cols[0]+"="+row[0]  
    elif len(cols) == 2:
        d = "DELETE FROM "+table+" WHERE "+cols[0]+"="+row[0]+" AND "+cols[1]+"="+row[1]
    else:
        d = "DELETE FROM "+table+" WHERE "+cols[0]+"="+row[0]+" AND "+cols[1]+"="+row[1]+" AND "+cols[2]+"="+row[2]
    try:
        cur.execute(d)
        print('Deleted:', d)
        con.commit()
    except:
        print(d, "\nЧто-то пошло не так. Проверьте, что введённые данные корректны, и запустите заново.")

Сейчас вам нужно будет выбрать, по каким полям из pet_id, name, kind, gender, age, owner_id нужно будет искать данные (не более трёх). Перечислите названия через запятую, пожалуйста.
По каким полям? L8-0987, simba, cat
Введите значения параметра L8-0987: pet_id
Введите значения параметра simba: name
Введите значения параметра cat: kind
DELETE FROM pets WHERE L8-0987=098014 AND simba=nod AND cat=cat 
Что-то пошло не так. Проверьте, что введённые данные корректны, и запустите заново.


#### Узнать pet_id питомца

In [40]:
name = input("Введите имя питомца: ")
kind = input("Какой у вас питомец? ")
age = input("Введите возраст питомца: ")

Введите имя питомца: Tiger
Какой у вас питомец? Dog
Введите возраст питомца: 15


In [42]:
get_pet_id = "SELECT pet_id FROM pets WHERE name='"+name+"' AND kind='"+kind+"' AND age='"+age+"'"
cur.execute(get_pet_id)
print("pet_id ващего питомца:", cur.fetchone()['pet_id'])

pet_id ващего питомца: N8-0553


#### Узнать owner_id владельца

In [43]:
name = input("Ваше имя: ")
surname = input("Ваша фамилия: ")
city = input("Ваш город: ")

Ваше имя: Carmen
Ваша фамилия: Ingram
Ваш город: Detroit


In [44]:
get_owner_id = "SELECT owner_id FROM owners WHERE name='"+name+"' AND surname='"+surname+"' AND city='"+city+"'"
cur.execute(get_owner_id)
print("owner_id владельца:", cur.fetchone()['owner_id'])

owner_id владельца: 9614


### Модификация данных

Как инструмент для исправления опечаток, которые часто встречаются в таких документах, например. Для модификации необходимо знать id питомца или владельца (в зависимости от запроса).

In [45]:
choice = input('Выберите таблицу, в которой хотите изменить что-то: pets (наберите 1), owners (2): ').lower()
if choice == '1' or choice == 'pets':
    table = 'pets'
    print('Вы выбрали таблицу', table)
elif choice == '2' or choice == 'owners':
    table = 'owners'
    print('Вы выбрали таблицу', table)
else:
    print('Вы ввели некорректный номер таблицы, попробуйте запустить заново.')

Выберите таблицу, в которой хотите изменить что-то: pets (наберите 1), owners (2): 2
Вы выбрали таблицу owners


In [46]:
if table == 'pets':
    pet_id = input("Введите pet_id вашего питомца: ")
else:
    owner_id = input("Введите owner_id: ")

Введите owner_id: 9614


In [47]:
if table == 'pets':
    what = input("Какое поле вы хотите поменять? ")
    check = "SELECT "+what+" FROM "+table+" WHERE pet_id='"+pet_id+"'"
    try:
        cur.execute(check)
        print("Сейчас это поле имеет значение", cur.fetchone()[what])
    except:
        print("Что-то пошло не так. Проверьте, существует ли такое поле.")
        raise Error
    goal = input("На что вы хотите его поменять? ")
    change = "UPDATE "+table+" SET "+what+"='"+goal+"' WHERE pet_id='"+pet_id+"'"
    try:
        cur.execute(change)
        print("Changed:", change)
    except:
        print("Что-то пошло не так во время изменения данных. Проверьте, корректен ли ваш запрос.")
elif table == 'owners':
    what = input("Какое поле вы хотите поменять? ")
    check = "SELECT "+what+" FROM "+table+" WHERE owner_id='"+owner_id+"'"
    try:
        cur.execute(check)
        print("Сейчас это поле имеет значение", cur.fetchone()[what])
    except:
        print("Что-то пошло не так. Проверьте, существует ли такое поле.")
    goal = input("На что вы хотите его поменять? ")
    change = "UPDATE "+table+" SET "+what+"='"+goal+"' WHERE owner_id='"+owner_id+"'"
    try:
        cur.execute(change)
        print("Changed:", change)
    except:
        print("Что-то пошло не так во время изменения данных. Проверьте, корректен ли ваш запрос.")
con.commit()

Какое поле вы хотите поменять? surname
Сейчас это поле имеет значение Ingram
На что вы хотите его поменять? Waterhouse
Changed: UPDATE owners SET surname='Waterhouse' WHERE owner_id='9614'


### Поиск

#### Поиск истории лечения

In [13]:
def search():
    result = None
    
    print('Введите информацию для поиска. Любое поле можно пропустить, нажав Enter.')

    info = {'pets.pet_id': 'Введите уникальный номер животного: ',
            'owners.owner_id': 'Введите уникальный номер владельца: ',
            'owners.surname': 'Введите фамилию владельца: ',
            'owners.city': 'Введите название города, в котором проходили лечение: '}

    query = f"""SELECT owners.owner_id, owners.city, owners.name, owners.surname, 
            pets.pet_id, pets.name, 
            history.date, details.type, details.subcode, details.description 
            FROM vet_database.owners 
            INNER JOIN vet_database.pets ON pets.owner_id = owners.owner_id
            INNER JOIN vet_database.history ON history.pet_id = pets.pet_id
            INNER JOIN vet_database.details ON details.id_proc = history.proc_id"""
    
    for k, v in info.items():
        ans = input(v)   
        try:
            q = query + f" WHERE {k} = '{ans}'"
            cur.execute(q)
            result = cur.fetchall()
        except:
            pass
        if result:
            return pd.DataFrame(result)
    print('Не удалось ничего найти.')
search()

Введите информацию для поиска. Любое поле можно пропустить, нажав Enter.
Введите уникальный номер животного: T0-3277


,owner_id,city,name,surname,pet_id,date,type,subcode,description
0,8133,Southfield,Humbert,Randolph,T0-3277,2016-01-22,VACCINATIONS,05,Rabies


In [14]:
search()

Введите информацию для поиска. Любое поле можно пропустить, нажав Enter.
Введите уникальный номер животного: 
Введите уникальный номер владельца: 
Введите фамилию владельца: 
Введите название города, в котором проходили лечение: Southfield


,owner_id,city,name,surname,pet_id,date,type,subcode,description
0,8133,Southfield,Humbert,Randolph,T0-3277,2016-01-22,VACCINATIONS,05,Rabies
1,9900,Southfield,Dior,Floyd,I2-7926,2016-05-28,VACCINATIONS,05,Rabies
2,1070,Southfield,Biscuit,Velazquez,U8-6473,2016-07-10,VACCINATIONS,05,Rabies
3,1070,Southfield,Biscuit,Velazquez,U8-6473,2016-08-19,GROOMING,01,Bath
4,1546,Southfield,Blackie,Blow,U6-4890,2016-10-19,VACCINATIONS,05,Rabies
5,7846,Southfield,Scout,Warren,U4-9376,2016-12-10,VACCINATIONS,05,Rabies


#### Поиск по дате обращения

In [30]:
def search_date():
    mi = """SELECT history.date FROM vet_database.history
            WHERE history.date = (SELECT MIN(history.date) FROM history)"""
    
    ma = """SELECT history.date FROM vet_database.history
            WHERE history.date = (SELECT MAX(history.date) FROM history)"""
    
    cur.execute(mi)
    mi = cur.fetchone()['date']
    cur.execute(ma)
    ma = cur.fetchone()['date']
    print('Самая ранняя дата: ', mi, '\n', 'Самая поздняя дата: ', ma)
    
    try:
        date = datetime.strptime(input('Введите дату в формате YYYY-MM-DD (например, 2016-02-05): '), '%Y-%m-%d')
    except:
        return 'Проверьте, правильно ли вы написали дату.'
        
    if date != '' and date >= mi and date <= ma:
        date2 = input('Введите вторую границу временного отрезка в формате YYYY-MM-DD \
        (например, 2016-02-05) или нажмите Enter: ')
        if date2:
            try:
                date2 = datetime.strptime(date2, '%Y-%m-%d')
            except:
                return 'Проверьте, правильно ли вы написали дату.'
         
            if date2 < date:
                date2, date = date, date2
           
            query = f"""SELECT date, pet_id, type, subcode, description, price 
                        FROM history
                        INNER JOIN details ON history.proc_id = details.id_proc
                        WHERE date >= '{date}' AND date <= '{date2}'"""
                                  
        else:
            first = input('Искать позже этой даты? "да" или "нет": ')
            if first.lower() == 'да':
                sign = '>='
            else:
                sign = '<='
            
            query = f"""SELECT date, pet_id, type, subcode, description, price 
                    FROM history
                    INNER JOIN details ON history.proc_id = details.id_proc
                    WHERE history.date {sign} '{date}'"""
            print(query)
    else:
        return 'В этот день в поликлинике ничего не происходило.'
                                  
    try:
        cur.execute(query)
        result = cur.fetchall()
        return pd.DataFrame(result)
    except:
        print('Проверьте, правильно ли вы ввели дату.')

In [31]:
search_date()

Самая ранняя дата:  2016-01-10 00:00:00 
 Самая поздняя дата:  2016-12-31 00:00:00
Введите дату в формате YYYY-MM-DD (например, 2016-02-05): 2016-09-12
Введите вторую границу временного отрезка в формате YYYY-MM-DD         (например, 2016-02-05) или нажмите Enter: 2016-09-12


,date,pet_id,type,subcode,description,price
0,2016-09-12,B1-7289,VACCINATIONS,05,Rabies,10
1,2016-09-12,D1-1113,GROOMING,03,Flea Spray,10
2,2016-09-12,C1-2139,GROOMING,03,Flea Spray,10


## Примеры более сложных запросов

### SELECT с несколькими таблицами

Такой запрос сводит информацию о владельце, питомце и процедурах из нескольких таблиц

In [32]:
query = """SELECT owners.name, owners.surname, pets.name, history.date, details.description
            FROM owners, pets, history, details
            WHERE owners.owner_id = pets.owner_id
            AND history.pet_id = pets.pet_id
            AND history.proc_id = details.id_proc"""

cur.execute(query)
sample = cur.fetchmany(size=5)
pd.DataFrame(sample)

,name,surname,date,description
0,Humbert,Randolph,2016-01-22,Rabies
1,Jake,McKenzie,2016-01-29,Rabies
2,Cuddles,Peterson,2016-01-30,Pinning-I.M.
3,Bandit,Tea,2016-02-02,Rabies
4,Cuddles,Reed,2016-02-04,Rabies


### Вложенные SELECT

Этот запрос выводит описание самой популярной процедуры, которую предлагают в клинике.

In [25]:
query = """SELECT details.type, details.description 
            FROM details 
            WHERE details.id_proc in 
            (SELECT proc_id FROM 
                    (SELECT proc_id, count(*) as topcount
                    FROM vet_database.history, vet_database.details
                    GROUP BY history.proc_id
                    ORDER BY topcount DESC
                    LIMIT 5) as foo)"""

cur.execute(query)
sample = cur.fetchmany(size=5)
pd.DataFrame(sample)

,type,description
0,VACCINATIONS,Rabies
1,GROOMING,Bath
2,GROOMING,Flea Spray
3,GENERAL SURGERIES,Declaw
4,VACCINATIONS,Galaxie (DHLPP)


Этот запрос позволяет узнать, какие процедуры делались в первый день периода.

In [26]:
query = """SELECT history.date, details.description, pets.kind
            FROM history, details, pets
            WHERE history.date = 
            (SELECT MIN(history.date) 
            FROM history)"""

cur.execute(query)
sample = cur.fetchmany(size=5)
pd.DataFrame(sample)

,date,description,kind
0,2016-01-10,Office Call,Dog
1,2016-01-10,Emergency,Dog
2,2016-01-10,Reck,Dog
3,2016-01-10,Bath,Dog
4,2016-01-10,Flea Dip,Dog


### Запросы с JOIN

Здесь можно посмотреть клиентов, которые потратили больше всего денег в поликлинике.

In [16]:
query = """SELECT owners.owner_id, owners.name, owners.surname, pets.name, history.date, SUM(details.price) as money
FROM vet_database.owners
INNER JOIN vet_database.pets ON pets.owner_id = owners.owner_id
INNER JOIN vet_database.history ON history.pet_id = pets.pet_id
INNER JOIN vet_database.details ON details.id_proc = history.proc_id
GROUP BY owners.owner_id
ORDER BY money DESC"""

cur.execute(query)
sample = cur.fetchmany(size=5)
pd.DataFrame(sample)

,owner_id,name,surname,date,money
0,8316,Bruce,Fay,2016-10-22,450
1,5806,Cookie,Reyna,2016-08-02,400
2,7579,Cuddles,Peterson,2016-01-30,325
3,1915,Rumba,Bowers,2016-12-01,305
4,7581,Vuitton,Nolen,2016-03-18,185


Этим запросом можно получить информацию о том, каким из животных в выбранном городе проводилась вакцинация.

In [19]:
query = """SELECT pets.pet_id, pets.name, pets.kind, details.type, owners.city 
            FROM vet_database.pets
            INNER JOIN vet_database.owners ON owners.owner_id = pets.owner_id
            INNER JOIN vet_database.history ON history.pet_id = pets.pet_id
            INNER JOIN vet_database.details ON details.id_proc = history.proc_id
            WHERE details.type = 'VACCINATIONS' 
            AND owners.city = 'Southfield'"""

cur.execute(query)
sample = cur.fetchmany(size=5)
pd.DataFrame(sample)

,pet_id,name,kind,type,city
0,T0-3277,Humbert,Cat,VACCINATIONS,Southfield
1,I2-7926,Dior,Parrot,VACCINATIONS,Southfield
2,U8-6473,Biscuit,Dog,VACCINATIONS,Southfield
3,U6-4890,Blackie,Dog,VACCINATIONS,Southfield
4,U4-9376,Scout,Dog,VACCINATIONS,Southfield


Этим запросом можно вывести информацию об истории лечения для каждого животного.

In [20]:
query = """SELECT owners.owner_id, owners.city, owners.name, owners.surname, 
            pets.pet_id, pets.name, 
            history.date, details.type, details.subcode, details.description 
            FROM vet_database.owners 
            INNER JOIN vet_database.pets ON pets.owner_id = owners.owner_id
            INNER JOIN vet_database.history ON history.pet_id = pets.pet_id
            INNER JOIN vet_database.details ON details.id_proc = history.proc_id"""

cur.execute(query)
sample = cur.fetchmany(size=5)
pd.DataFrame(sample)

,owner_id,city,name,surname,pet_id,date,type,subcode,description
0,8133,Southfield,Humbert,Randolph,T0-3277,2016-01-22,VACCINATIONS,05,Rabies
1,3089,Marquette,Jake,McKenzie,G6-6501,2016-01-29,VACCINATIONS,05,Rabies
2,7579,Ann Arbor,Cuddles,Peterson,P9-3625,2016-01-30,ORTHOPEDIC,07,Pinning-I.M.
3,9604,Roseville,Bandit,Tea,F1-1855,2016-02-02,VACCINATIONS,05,Rabies
4,5447,Lansing,Cuddles,Reed,S1-2243,2016-02-04,VACCINATIONS,05,Rabies
